In [4]:
from __future__ import print_function
from keras.models               import Sequential, load_model
from keras.layers               import Input, Dense, Activation
from keras.layers               import LSTM, GRU, SimpleRNN
from keras.optimizers           import RMSprop, Adam
from keras.utils.data_utils     import get_file
from keras.layers.normalization import BatchNormalization as BN
from keras.layers.noise         import GaussianNoise as GN
from keras.layers.noise         import GaussianDropout as GD
import numpy as np
import random
import sys
import tensorflow               as tf 
tf.logging.set_verbosity(tf.logging.ERROR)
import glob
import json
import pickle
#import msgpack
#import msgpack_numpy as mn
#mn.patch()
#import MeCab
#import plyvel
from itertools import cycle as Cycle
#import dill
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import util
from sklearn import metrics

Using TensorFlow backend.


In [5]:
import pickle
emoji_index = pickle.loads(open('./emoji_index.pkl', 'rb').read())
def emojiFromIndex(num):
    return [k for k, v in emoji_index.items() if v == num]

emojiFromIndex(10)

['💦']

In [6]:
def plot_history(history, num):
    # 精度の履歴をプロット
    plt.plot(history.history['acc'],"o-",label="accuracy")
    plt.plot(history.history['val_acc'],"o-",label="val_acc")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc="lower right")
    plt.show()

    # 損失の履歴をプロット
    plt.plot(history.history['loss'],"o-",label="loss",)
    plt.plot(history.history['val_loss'],"o-",label="val_loss")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='lower right')
    plt.savefig("./graph/keras_graph_%d" % num)

In [7]:
def discriminator_model():
    from keras.layers import Input, Dense, Embedding, merge, Convolution2D as Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
    from keras.layers.core import Reshape, Flatten
    from keras.models import Model, load_model
    from keras.layers.merge import add, concatenate
    sequence_length = 30
    embedding_dim = 200  # 100
    vocabulary_size = 10
    num_filters = 512
    filter_sizes = [3, 4, 5, 1, 2]
    inputs = Input(shape=(sequence_length, embedding_dim,), dtype='float64')
    #embedding = Embedding(output_dim=embedding_dim, input_dim=vocabulary_size, input_length=sequence_length)(inputs)
    reshape = Reshape((sequence_length, embedding_dim, 1))(inputs)
    #Conv2D(512, (3, 256), padding="valid", kernel_initializer="normal", data_format="channels_last", activation="relu")

    conv_0 = Conv2D(num_filters, (filter_sizes[0], embedding_dim),
                    kernel_initializer="normal", data_format="channels_last", activation="relu")(reshape)
    conv_1 = Conv2D(num_filters, (filter_sizes[1], embedding_dim),
                    kernel_initializer="normal", data_format="channels_last", activation="relu")(reshape)
    conv_2 = Conv2D(num_filters, (filter_sizes[2], embedding_dim),
                    kernel_initializer="normal", data_format="channels_last", activation="relu")(reshape)
    conv_3 = Conv2D(num_filters, (filter_sizes[3], embedding_dim),
                    kernel_initializer="normal", data_format="channels_last", activation="relu")(reshape)
    conv_4 = Conv2D(num_filters, (filter_sizes[4], embedding_dim),
                    kernel_initializer="normal", data_format="channels_last", activation="relu")(reshape)
    # `MaxPooling2D(pool_size=(28, 1), padding="valid", data_format="channels_last", strides=(1, 1))
    maxpool_0 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(
        1, 1), padding="valid", data_format="channels_last")(conv_0)
    maxpool_1 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(
        1, 1), padding="valid", data_format="channels_last")(conv_1)
    maxpool_2 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(
        1, 1), padding="valid", data_format="channels_last")(conv_2)
    maxpool_3 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[3] + -3, 1), strides=(
        1, 1), padding="valid", data_format="channels_last")(conv_3)
    maxpool_4 = MaxPooling2D(pool_size=(sequence_length - filter_sizes[4] + -2, 1), strides=(
        1, 1), padding="valid", data_format="channels_last")(conv_4)

    merged = concatenate(
        [maxpool_0, maxpool_1, maxpool_2, maxpool_3, maxpool_4], axis=1)
    
    #units = 2048
    output = Dense(units=1, activation='sigmoid')(
        Activation('linear')(
            Dropout(0.5)(
                Flatten()(merged))))

    model = Model(inputs=inputs, outputs=output)
    adam = Adam()
    model.compile(optimizer=adam, loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


In [8]:
dataloader = util.dataloader(dataPath='../text_vec.pkl', labelPath='../label_vec.pkl')
dataloader.normalize()
X_train, X_test, y_train, y_test = dataloader.dataset() 


sample_nums: [(2, 53592), (10, 38038), (9, 26034), (1, 22109), (4, 17432), (5, 16174), (7, 13489), (3, 10927), (6, 10851), (8, 9534)]
min_num: 9534
クラス2 削除サンプル数: 44058 (82.21％)
クラス10 削除サンプル数: 28504 (74.94％)
クラス9 削除サンプル数: 16500 (63.38％)
クラス1 削除サンプル数: 12575 (56.88％)
クラス4 削除サンプル数: 7898 (45.31％)
クラス5 削除サンプル数: 6640 (41.05％)
クラス7 削除サンプル数: 3955 (29.32％)
クラス3 削除サンプル数: 1393 (12.75％)
クラス6 削除サンプル数: 1317 (12.14％)
クラス8 削除サンプル数: 0 (0.00％)


In [9]:
def main_train():
    print('Vectorization...')
#     X = np.zeros((len(X_train), 30, 200), dtype=np.float64)
#     y = np.zeros((len(X_train), 2048), dtype=np.float64)
#     for i, (text, ans) in enumerate(zip(X_train, y_train)):
#         if i%10000 == 0:
#             print("building training vector... iter %d"%i)
#         for t, term in enumerate(text):
#             X[i, t, :] = term
#         y[i, :] = ans
    model = discriminator_model()
    for iteration in range(1, 5):
        #for iteration in range(1, 50):
        print()
        print('-' * 50)
        print('Iteration', iteration)
        history = model.fit(X_train, y_train, batch_size=128, epochs=20, validation_split=0.33)
        MODEL_NAME = "./models/snapshot.%09d.model"%(iteration)
        model.save(MODEL_NAME)
        plot_history(history, iteration)


In [10]:
def pred():
    model_type = sorted(glob.glob('./models/*.model'))[-1]
    print("model type is %s"%model_type)
    model  = load_model(model_type)
    print("finished model type is %s"%model_type)
    #results = model.predict(np.array([X_test]))
    y_pred = model.predict_classes(X_test, batch_size=128, verbose=1)
    
    target_names = [str(i) for i, e in emoji_index]
    report = metrics.classification_report(y_test, y_pred, target_names=target_names)
    print(report)
    
    matrix = metrics.confusion_matrix(y_test, y_pred)
    print(matrix)

In [11]:
main_train()

Vectorization...

--------------------------------------------------
Iteration 1
Train on 116944 samples, validate on 57600 samples
Epoch 1/20
  1152/116944 [..............................] - ETA: 53:56 - loss: -53.2457 - acc: 0.0955  

/usr/local/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.276111). Check your callbacks.
  % delta_t_median)


116944/116944 [==============================] - 54s 460us/step - loss: -68.8079 - acc: 0.1010 - val_loss: -69.0535 - val_acc: 0.1011
Epoch 2/20
116944/116944 [==============================] - 20s 172us/step - loss: -69.1711 - acc: 0.1012 - val_loss: -69.0535 - val_acc: 0.1011
Epoch 3/20
116944/116944 [==============================] - 20s 171us/step - loss: -69.1711 - acc: 0.1012 - val_loss: -69.0535 - val_acc: 0.1011
Epoch 4/20
116944/116944 [==============================] - 20s 174us/step - loss: -69.1711 - acc: 0.1012 - val_loss: -69.0535 - val_acc: 0.1011
Epoch 5/20
116944/116944 [==============================] - 20s 173us/step - loss: -69.1711 - acc: 0.1012 - val_loss: -69.0535 - val_acc: 0.1011
Epoch 6/20
102400/116944 [=========================>....] - ETA: 2s - loss: -69.1932 - acc: 0.1014

KeyboardInterrupt: 

In [12]:
def pred():
    #m = MeCab.Tagger('-Owakati')
    m = MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')
    emoji_index = pickle.loads(open('./emoji_index.pkl', 'rb').read())
    index_emoji = {index:emoji for emoji, index in emoji_index.items()}
  
    print('now loading term_vec.pkl...')
    term_vec    = pickle.loads(open('term_vec.pkl', 'rb').read())
    print('finished loading term_vec.pkl...')
    #model_type = sorted(glob.glob('./models/*.model'))[-1]
    model_type = sorted(glob.glob('./models/*.model'))[-1]
    print("model type is %s"%model_type)
    model  = load_model(model_type)
    print("finished model type is %s"%model_type)
    for line in sys.stdin:
        line = line.strip()
        print(line, end=" ")
        buff = ["*"]*30
        for i,term in enumerate(m.parse(line).strip().split()[:30]):
            buff[i] = term
        X = []
    for term in buff:
        if term_vec.get(term) is not None:
            X.append(term_vec[term])
        else:
            X.append(np.zeros(256))#100
    results = model.predict(np.array([X]))
    res = {index_emoji[i]:score for i,score in enumerate(results[0].tolist()[:1187])}
    for emoji, score in sorted(filter(lambda x:x[1]>0.01, res.items()), key=lambda x:x[1]*-1)[:20]:
        print(emoji, "%d"%(int(score*100)), end=" ")
    print()

In [14]:
import MeCab 
pred()

RuntimeError: 